# Gold Team : Optimisation of Circuit Configuration using Genetic Algorithm
### By Kevin Fung, Mattia Guerri, Tayfun Karaderi, Chirayu Khimji, Deirdree Polak and Laura Su

This report outlines the algorithms used to optimise the circuit configuration of a separation process factory. A clear explanation of the choice of parameters is also given, for optimal performance. Finally a parallelised version of the software is presented.

# Table of Content

## 1 - Solution Algorithm

## 2 - Results

## 3 - Convergence Behaviour and Model Parameters

## 4 - Testing

## 5 - Parallelisation

## 6 - Optimisation

## 7 - Further Development

## 8 - Conclusion

# 1 - Solution Algorithm

## Structure

The code is cleverly organized around three main classes: Genetic_Algorithm, Circuit and Validity. The code is run from the main and only requires key parameters such as number of units, number of genrated circuit, probability of mutation, etc. 

The Genetic Algorithm object is created from the main, and the runAlgo method is used. Within that function, the Generate Circuit is run, along with Validity object and methods, and the Circuit Evaluation. 

The following descriptions runs through the main steps the Circuit Congiguration program, more precisely the runAlgo method, goes through in order to optimise the units layout. 


## Step One : Random Circuit Generation 

First, an initial ensemble of  circuits is produced. The circuits are defined by vectors of integers. The first number indicates the feed. Then, each pair of numbers indicate the direction of the concentrate (first number in the pair) and the tail (second number in the pair) for each unit. The numbers are drawn randomly, and a certain number of random circuits are created.

At this stage already the circuits are verified using the Validity object. The circuits are generated iteratively until a certain number of circuits (a 100 for example) is obtained. Once all circuits have been created, a fitness value is obtained through the evaluation.

## Step Two : Circuit Validation

Randomly generating circuit vectors results in many vectors that have a low performance. For this reason, a pre-selection is done to the random vectors generated and if they don't meet a certain criteria which are indicative of bad performance then they are regenerated. This way the genetic algorithm can converge to a good solution with a smaller sample size.

In order to validate the circuit, a Validity object is created using only the size of the circuit. Each circuit generated can then be passed in the Check_Validity method individually.

There are four criteria used to validate a circuit:

- There should be no self-recycle
- The destination for both products from a unit should not be the same (different) unit.
- Every unit must be accessible from the feed
- Every unit must have a route forward to both of the outlet streams.


Criteria 1 and 2 were simply tested by looping over the circuit vector -- indexing unit id with iteration number and returning False if 1 of the streams lead to its own id or if 2 streams lead to the same unit.

Criteria 4 were tested using a recursive approach. We start from a unit's concentrate stream and check that it goes to an outlet. We iterate until an end is seen and if an end is seen we set the value of the global variable seen_conc to True. We do the same for the tail stream and if one of seen_conc or seen_tail variables are false starting from any unit, then we return False. Criteria 3 is a very similar approach to 4 but this time checks that all units can be seen starting from unit 0.

### Example of a valid circuit

<img src="report_pic/output_tayfun4.png" style="width: 400px">


### Examples of invalid circuits

This circuit vector is invalid because unit 3 self-cycles and doesnt see the tailing outlet:

<img src="report_pic/output_tayfun1.png" style="width: 400px">

The above circuit is also invalid as unit 3 doesn't see the feed:

<img src="report_pic/output_tayfun2.png" style="width: 400px">

Also, invalid as units 1 and 2 doesnt see concentrate outlet and unit 3 doesnt see the tailings outlet. Also, 3 recycles:
<img src="report_pic/output_tayfun3.png" style="width: 400px">

The Circuit_Validate(circuit_vector) function implimented takes a vector type circuit vector and returns a boleen value of True for valid and False for invalid circuits. The functions were tested with the above examples and verified that it works for these test cases (i.e. as expected returns True for the first case and False for the others).


## Step Three : Circuit Evaluation

The solution depends on three classes: the Circuit class, CUnit class, and CStream class. The CStream class contains the mass flow of the stream components and the necessary operation methods for incrementing and adding streams. The CUnit class contains relevant CStream objects pertaining to their feed and destination streams and methods manipulating the streams for the Circuit Evaluation algorithm.

To evaluate the circuit, the Circuit class must be instantiated to create a new Circuit object, the user is given the choice to input their own performance costs and input circuit feeds or to keep the default problem parameters. A single function "Evaluate Circuit" is called which takes in a vector of the circuit configuration and convergence parameters.
The circuit class contains a vector of unit objects pertaining to the circuit configurations as well as parameters for the circuit evaluation algorithm.

### Algorithm
The function updates unit's content and calculates the economic performance of the whole circuit using a Jacobian like Iterative method:

1- Fill all units with a the feed values given for valuable and waste component (for example, 10kg/s and 100kg/s) for one second.

While the relative difference is less than tolerance, and the total number of time steps is below a certain threshold (until convergence). This assumes time step is one second throughout. 

2- Transfer the valuable and waste components in each unit in the concentrate and tail streams. The total amount going into each stream can be deducted using the concentrate component fraction (in this case, 20% of valuable and 5% of waste components)

3- Reset the input feed to 0, but save the old feed in a variable. Also peform this operation into the two end streams going into the concentrate and trailing bins. This will be used to calculate the relative feed difference.

4- Send the concentrate and trail streams calculated in step 2 to their destination units.

5- Calculate the relative difference for the two feeds of each unit, using the feed values for each unit claclulated in step 4. Take the largest relative difference for stream valuable and stream waste from all the units.

$$ {rel\_tol} = \frac{feed - {old feed}}{{old feed}} $$

6- If relative difference is larger than the set tolerance, break the while loop, otherwise run step 2 to 5 again.

7- Calculate the performance criteria from the amount of valuable and waste componenent collected in the concentrate bin. If there was no convergence, and the relative difference remained larger then the tolerance through all the time steps, the fitness performance criteria is set to the worst possible value: the total cost of waste (in this case -500£) for the input feed rate (in this case 100kg/s)

$$ Fitness = \left(total\,amount\,of\,valuable\,*\,price\,of\,valuable \right) + \left(total\,amount\,of\,waste\,*\,cost\,of\,waste \right) $$

### Discussion

Optimisation of the code was kept in mind. We avoid using if statements in our loops for setting specific values for the circuit output streams in our vector lists and instead directly set the specific values via indexing. This is to avoid using unnecessary if operations in the loop and saves time.

To include the output streams in the circuit for evaluation, we include two additional unit objects to contain these streams. In addition, we realised we had to set the correct unit which would recieve the input circuit feed, a problem which we quickly overcame. Finally, we also had to reset the output stream units to achieve convergence as well.


## Step Four : Genetic Algorithm

Once the generated circuits have been tested and evaluated, the genetic algorithm can be called. This algorithm is in the geneticAlgo method, and returns the best circuit possible.

### Algorithm

After initialising the list of parent vectors as described above, these vectors and the vector of fitness values are fed in the geneticAlgo function, as well as the desired max iteration and other inputs for the Evaluation_circuit function.
This geneticAlgo function was implemented following the steps below:

1) The circuit with the best fitness goes through the next generation unchanged, as calculated in the Circuit Evaluation section using the Evaluation_Circuit function.

2) Two circuits are randomly chosen. The probability of each circuit to be chosen is proportional to its fitness.

3) There is about a 20 % probability for the two vectors to cross-over each other and generate two children circuits. If no cross-over happens, the circuits go to the next step unchanged.

4) There is a small probability (less than 1%) for each component of the two circuits to mutate. When mutating each value, a randomly picked number replace the mutated value. The mutated number cannot be higher than number of units - 1 (for the feed) or higher than number of units + 1 (for all the other elements). To make sure the mutations are within these limits, we take the remainder of the mutated number divided by number of units - 1 or number of units + 1 for the feed and the other elements, respectively.

5) We  check   the  validity  of  the  obtained   circuits  and,  if   valid,  add   them  to  the  next generation.

6) These steps are repeated until enough circuits for the next generation are obtained.

7) The parents vectors are replaced with the children and step 6 is repeated until either a number maxIte of iterations or the best vector has not changed for a sufficient number of iterations bestIndCnt. The convergence criteria are implemented in the code as:
- the maximum number of iterations maxIte
- the minimum number of iterations to be done minIte
- the minimum number of iterations bestIndCnt for which the best circuit stays the same, and can end the code if this number is reached 


### Discussion

Two different designs were tested in developing the genetic algorithm:

-    Implementing a class containing a circuit vector and its fitness value
-    “Basic” implementation: vector of integers, vector of those vectors and list of fitness values

In the first case, there were some memory problems and as it wasn’t more difficult to store all of the fitness values in one vector, we decided to do the second option.


# 2 - Results for base case

## Investigating the best parameters to use

The code has many parameters which can change the overall performance. They can be classified as the convergence criteria of the genetic algorithm, i.e. maxIte, minIte and bestIndCnt presented above, and as the other parameters, i.e. the probability of cross-over proCrosOver, the probability of mutation proMut and the number of child vectors num_circuits. 

The algorithm was run changing the probability of cross-over, of mutation and the number of child vectors separately. 
<br>
The convergence criteria were modified according to the number of units used and kept constant over the runnings. We had to make sure that the algorithm finishes and that he will converge to the best value of the case. After try and error, we decided to fix the maximum number of iterations to 10000, the minimum number of iterations to 0 and the minimum number of iterations for which the best circuit stays the same to 2500.


The results are represented in the graphs below.

<img src="report_pic/best_fit_crossover.png" style="width: 400px">
<img src="report_pic/best_fit_mutation.png" style="width: 400px">
<img src="report_pic/best_fit_num_children.png" style="width: 400px">

<img src="report_pic/time_taken_crossover.png" style="width: 400px">
<img src="report_pic/time_taken_mutation.png" style="width: 400px">
<img src="report_pic/time_taken_num_children.png" style="width: 400px">


As expected, increasing the number of child vectors slows down the algorithm. At around a number of 100, the fitness value is the best.

The cross-over graphs show generally that having a probability close to 1 gives the best overall performance, in terms of speed and of finding the best fitness value (in this case the optimum seems to be 0.8).

The mutation results show that a value of probability relatively close to 0.01 would give the best results and performance.


## Best Circuit

The optimum circuit configuration used for 10 units is the following:
- num_circuits = 100 
- proCrosOver = 0.8
- proMut = 0.005
- maxIte = 10000
- minIte = 0
- bestIndCnt = 2500

One possible representation of the best vector obtained is {9, 5, 4, 2, 7, 5, 0, 2, 1, 5, 9, 10, 2, 4, 11, 0, 6, 2, 3, 2, 8}.

This is picture below. **Its fitness value is 165.748.**

<img src="report_pic/output.jpg" style="width: 800px">

On an average of 15 runs of the code, the best fitness value is almost always 165.748, with an average number of iterations of about 3000.

For this configuration, it takes in average about 8 seconds to run until there is convergence. The fastest convergence takes about 5 seconds, and the slowest can take about 15 seconds.


## Performance analysis

Most of the time, the algorithm is either running faster in terms of speed but affecting the end value (which would then not be the very best), or giving a better value by waiting more time (changing the convergence parameters notably). Some optimum can be achieved by balancing out both.

Since for circuits with more units, it will take way more time to run, we would really need to compromise between getting one of the best values and the time taken to run the algorithm as finding the best value may be really time-consuming! 

Also, the generation of the parent vectors is random. Thus, there may be some cases where the genetic algorithm starts with either a "good" guess or with a "bad" guess. This can affect the convergence and the time taken to find the best vector and fitness value significantly. 

Note that the code was tested using Visual Studio 2017 on Release mode.

# 3 - Convergence Behaviour and Model Parameters

An investigation into the model parameters has been carried out. The theory follows that if we have a high concentration relative to waste price then we have a high performance fitting for the circuits produced in our algorithm. Thus, circuits with a bad recovery are found. We test this idea by varying the ratio of concentration to waste price ($ratio = \frac{conc}{waste}$) from 0 to 2 and measuring performance.

<img src="report_pic/priceratio.png" style="width: 800px">

From the figure above, we see that when waste price is much higher than concentrate price, we generate a better performing circuit than a lower waste price and high concentrate price. This is because the performance parameter in the Genetic Algorithm becomes more punishing if we have a higher waste price to concentrate ratio and thus produces circuits with better recovery. One recommendation would therefore to be to have a high waste price and low concentrate price when running the program.

To get the purity vs recovery graph, we ran the algorithm for different values of waste cost. What we want to show in this graph is the shape of the curve. This curve is a limiting curve as we can’t get better results than that in our configuration.

<img src="report_pic/purity_recovery.png" style="width: 800px">

We can see for example that when the waste cost is 0, we have the best recovery possible, but with a very bad purity. The best result we could hope in an economical aspect is a compromise between purity and recovery, as we would want to recover as much valuable material as possible with the best grade.

If we changed the feed rate, the current graph would be translating.

# 4 - Testing

Three tests were Developed: test_circuit.cpp, test_GA.cpp, test_Validity.cpp.
 
1. test_circuit.cpp passes a circuit, with a known true fitness value at a certain max_iter, into Evaluate_Circuit(example1, tol, max_iter) and then checks that difference between the computed fitness and the known fitness is less than the known tolerance.
 
2. test_GA checks whether the best vector previous generation is the first vector in the new generation. Our Algorithm passes this test:
```c++
            if (ga.circuits[0][j] != best_circuit[j])
                pass = false;```
 
3. Test_validity.cpp passes 3 known circuits into Check_Validity() in order to check whether or not the circuits are valid or not:
           
 
Improvements:
Check convergence rigorously and if there are problematic configurations then we could write new tests within Validity.cpp to check this

# 5 - Parallelisation

## Shared Memory Parallelisation - Open MP

### Circuit Validation

In order to parallelise the circuit validation and evaluation, it was decided to use shared memory and OpenMP. This was thought more efficient than distributed memory because a single circuit vector is passed in the function everytime, no communication is done, and most of the two functions need to run in serial. 

For Circuit Verification, the function was parallelised using the OMP parallel section directives. This way it was thought, all four tests within the function could run in parallell, thus dividing the time to run by approximately four.

However, when tested, it actually took longer to run. This was because, as the code is currently written, if the first test is found to be false, it return false immediately, without running the following three tests. When running the code parallel in section, all four tests needed to be run, then if the bool for any of the four tests is false, only then does the function return false. This was therefore found to take longer when tested with a large number of circuit_vectors, that were  valid and invalid. 

Moreover, the four section of code to be run are relatively short (except maybe for the recursive trasverse of the circuit) this means that actually setting up the threads for each section, and then closing those threads can take longer than running the program. 

```C++
bool Verification(vetor<int> circuit)
{
    //test1
    if (test1 = false)
        return false;
    
    ...
    
    //test4
    if (test4 = false)
        return false;
    
    return true;
      
}
```

In parallel this would be:

```C++
bool Verification(vetor<int> circuit)
{
#pragma omp parallel section
    {
        #pragma omp section
        {
            //test1
            if (pass)
                test1 = true;
            else test1 = false;
        }

        ...
        #pragma omp section
        {
            //test4
            if (pass)
                test4 = true;
            else test4 = false;
        }
    }
    
    if (!test1 || !test2 || !test3 || !test4) 
        return false
    else
        return true; 
}

```

The computing time after running the code with OMP section and without, for 10,000 circuit vectors is:
- 3.65s without OMP and with a return false structure (first code snippet)
- 7.89s without OMP and with a test 1= false, ... test 4 = false structure (second code snippet)
- 10.12s with OMP and with a test 1= false, ... test 4 = false structure (second code snippet)


### Circuit Evaluation

The five for loops within the evaluation algorithm were parallelised using OMP's parallel for directive. These loops don't need to communicate with eachother, they simply loop thorugh the unit list and change some values in each individual unit. Other directives were tried, for example, tasks, but it was found only the for loop could be parallelised safely, as the rest of the algorithm needed to be run in serial. The number of threads was set to 4. 

However, once parallelised the loops actually longer to run than in serial. Even when running guided scheduling, the default chunk size was to attribute all for loops to one thread only, which shows how inneficient it is to use OpenMp for the for loops. 

Again, the main reason for it is that it takes longer to open and close the threads than run the loop in parallel. Moreover, the loops are only the size of the circuit, so 10 or 15, which makes them too small to parallelise efficiently. 

The time efficiency is the following. These were recorded for 2 different sized vectors, and run 10 times to get the average. 

| Time       | 5 units    | 10 units|
|------------|------------|---------|
|without OMP |0.0227168   |0.0038155|
|with OMP    |0.1045518  |0.0485719|


### Genetic Algorithm

The genetic algorithm needs to be run in serial per circuit, therefore omp section or omp task cannot be efficiently used. There are a few for loops in the main algorithm. The first for loop updates the maximum value at each iteration, if the value calculated is bigger than the previous. It is therefore not easily parallelised, since each iteration needs to know the previous value, and there might be so misscomunnication if a thread runs faster than the other. Otherwise the code could have been altered to set the calculated as private and then compared after the for loop is finsihed. This however requires more code to be written to save the value on the stack, and therefore is not very efficient.

There are also numerous small loops, looping through the circuit vector values. However again, they were again too small, as proven for the circuit evaluation's loops to parallelise.

The large evaluation loop was parallelised using omp for loop with guided scheduling. This loops runs through all 100 of the circuit vectors, evaluates them and returns the fitness value. This seemed like an ideal candidate for parallelisation. However when running it, it was actually taking twice as long on average. 

For testing purposes, the maximum number of iterations was cut at 400. In that case, with omp the code run n average foe 16 seconds, while it only run in 8 seconds without omp. 


### Conclusion

Overall, while OpenMP was thought to be able to parallelise parts of different processes, it actually turned out inefficient in all cases. 

There are also options to parallelise the entire code using OpenMP, but it was agreed before testing it that using distributed memory to parallelise the whole process would be more efficient. 

The OMP code is still included in the OMP folder. 


## Distributed Memory - Message Passing Interface

### Circuit Validation and Evaluation

After analysis, it was decided not to parallelise the circuit evaluation and validation. This is because each function is called only passing in one circuit vector. Therefore it is not necessary to parallelise the content of the two functions, since they need to be run in serial.

### Genetic Algorithm

For the genetic Algorithm, two approaches were taken to parallelise the code using distributed memory with the MPI library:

First one runs the total number n of circuits (in base case it's a 100) in each process. Each node will generate their own circuits, and run the algorithm to completion. The master process will also work on its own group of circuits and when finished it will then gather the best circuit obtained by each process. The circuit with the best fitness is then output.

The second approach also runs the total number n of circuits on each process. The first process to end its iterations will send a message to all the other processes that will interrupt their calculation. This entails that the best circuit obtained by the processes that finishes first is taken as the best circuit.

A last approach was considered, but later abandoned. This was to split the number of circuit between the processors. However this requires a lot of communication between the processes and therefore will slow down the communication considerably, especially when the code is run only for 100 circuits.

# 6 - Further Developments

## Real Life Improvements

Mineral Speration rarely occurs in only 10 process, and even more rarely are there only two components(valuable and waste) to the flow. The code was therefore further developped and changed to include an option where the user can more precisely add in the number of components, their respective flow rate, concentrate sucess fraction and prices. 

The code run correctly and gives the same results as the main code. It is included in the extra_AdditionalComponents folder. It still needs to be optimised, but shows the possibilities available to expand the program.  

## Validity Check

An additional validity check should be added to ensure that the concentrate stream does not end up in the tailing outlet. This would make the circuit more efficient, and ensure no valuable material is lost to the tailing outlet. 

```C++
// Check no pairs are the same
	for (int i = 1; i < (circuit_vector.size() - 1); i += 2) {
		if (circuit_vector[i] == circuit_vector[i + 1]|| circuit_vector[i+1] == num_units - 1) {
			return false;
		}
	}
```

<img src="report_pic/Bad_Convergence_Example.png" style="width: 800px">

The fitness value is 121.  It can't escape from this configuration even after thousands of iterations.

# 7 - Conclusion

Overall the program runs smoothly and includes all of the optional tasks available. 

The best circuit, with 10 units, reaches the maximum value of 165. The parameters were tuned to ernsure the best one could be chosen. The program was also expanded to accept more than teo components. The program was optimised using diagnostic tools on Visual studio and the verification files were optimised considerably. 
